In [1]:
from __future__ import unicode_literals
import logging
import os
import re
import time
from urllib.parse import urlparse
import random

import pdfkit
import requests
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq


In [2]:
options = {
    'page-size':'Letter',
    'margin-top':'0.75in',
    'margin-right':'0.75in',
    'margin-bottom':'0.75in',
    'margin-left':'0.75in',
    'encoding':"UTF-8",
    'custom-header':[
        ('Accept-Encoding','gzip')
    ],
    'cookie':[
        ('cookie-name1','cookie-value1'),
        ('cookie-name2','cookie-value2'),
    ],
    'outline-depth':10,
}
htmls = []

html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
</head>
<body>
{content}
</body>
</html>
"""
user_agent_list = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]
#代理池https://www.kuaidaili.com/free/
proxyip = """139.199.38.177    8118  
125.117.129.201   9000  
115.218.221.1     9000  
47.97.8.244       3128  
183.166.129.53    8080  
221.206.100.133   54781 
125.40.238.181    56738 
111.230.237.187   8118  
120.26.199.103    8118  
114.234.81.201    9000  
112.243.88.228    9000  
119.145.136.126   8888  
180.118.86.242    9000  
117.90.252.252    9000  
222.135.92.68     38094"""
proxyip=proxyip.split()
proxyip = ["http://"+proxyip[x] + ":" + proxyip[x + 1] for x in range(len(proxyip)) if x % 2 == 0]
proxies = {'http':str(random.choice(proxyip))}
proxyip

['http://139.199.38.177:8118',
 'http://125.117.129.201:9000',
 'http://115.218.221.1:9000',
 'http://47.97.8.244:3128',
 'http://183.166.129.53:8080',
 'http://221.206.100.133:54781',
 'http://125.40.238.181:56738',
 'http://111.230.237.187:8118',
 'http://120.26.199.103:8118',
 'http://114.234.81.201:9000',
 'http://112.243.88.228:9000',
 'http://119.145.136.126:8888',
 'http://180.118.86.242:9000',
 'http://117.90.252.252:9000',
 'http://222.135.92.68:38094']

In [3]:
import random
print(len(user_agent_list))
random.choice(user_agent_list)


18


'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3'

In [4]:
name = "廖雪峰Python3"
start_url = "https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000"
domain = '{uri.scheme}://{uri.netloc}'.format(uri=urlparse(start_url))
headers = {
        'user-agent': random.choice(user_agent_list)
    }
response = requests.get(start_url,headers=headers)

In [5]:
def parse_menu(response):
    soup = BeautifulSoup(response.content,"html.parser")
    menu_tag = soup.find_all(class_="uk-nav uk-nav-side")[1]
    for li in menu_tag.find_all("div"):
        url = li.a.get("href")
        if not url.startswith("http"):
            url = "".join([domain,url])
        yield url

In [6]:
urls = [url for url in parse_menu(response)]

In [7]:
urls

['https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431608990315a01b575e2ab041168ff0df194698afac000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014316090478912dab2a3a9e8f4ed49d28854b292f85bb000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143161198846783e33de56d4041058c3dfc7e44ee1203000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431611988455689d4c116b2c4ed6aec000776c00ed52000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014316399410395f704750ee9440228135925a6ca1dad8000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432523496782e0946b0f454549c0888d05959b99860f000',
 'https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e

In [8]:
for index,url in enumerate(urls):
    print(index,url)

0 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000
1 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431608990315a01b575e2ab041168ff0df194698afac000
2 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014316090478912dab2a3a9e8f4ed49d28854b292f85bb000
3 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143161198846783e33de56d4041058c3dfc7e44ee1203000
4 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431611988455689d4c116b2c4ed6aec000776c00ed52000
5 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014316399410395f704750ee9440228135925a6ca1dad8000
6 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432523496782e0946b0f454549c0888d05959b99860f000
7 https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431

In [37]:
response1 = requests.get(url,headers=headers)

In [61]:
soup1 = BeautifulSoup(response.content,'html.parser')
body = soup1.find_all(class_="x-wiki-content x-main-content")[0]
title = soup1.find('h4').get_text()
center_tag = soup1.new_tag("center")
title_tag = soup1.new_tag('h1')
title_tag.string = title
center_tag.insert(1,title_tag)
body.insert(1,center_tag)
html = str(body)
pattern = "(<img .*?src=\")(.*?)(\")"

def func(m):
    if not m.group(2).startswith("http"):
        rtn = "".join([m.group(1),self.domain,m.group(2),m.group(3)])
        return rtn
    else:
        return "".join([m.group(1),m.group(2),m.group(3)])

In [62]:
html = re.compile(pattern).sub(func,html)
html = html_template.format(content=html)
html = html.encode("utf-8")

In [65]:
url503='https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014316090478912dab2a3a9e8f4ed49d28854b292f85bb000'
res503=requests.get(url,headers=headers)

In [70]:
res503

b'<html><body><h1>503 Service Unavailable for Bot</h1></body></html>'

In [126]:
proxyipinfo = """119.1.97.187	36751	高匿名	HTTP	贵州省黔西南布依族苗族自治州 电信	3秒	2018-12-11 21:30:59
61.184.109.33	50371	高匿名	HTTP	湖北省黄石市 电信	1秒	2018-12-11 20:30:56
115.223.250.211	9000	高匿名	HTTP	浙江省温州市 电信	3秒	2018-12-11 19:30:59
115.223.214.111	9000	高匿名	HTTP	浙江省温州市 电信	3秒	2018-12-11 18:30:55
119.145.2.101	44129	高匿名	HTTP	广东省深圳市 电信	1秒	2018-12-11 17:31:00
114.234.81.241	9000	高匿名	HTTP	江苏省徐州市 电信	1秒	2018-12-11 16:30:57
115.223.215.10	9000	高匿名	HTTP	浙江省温州市 电信	3秒	2018-12-11 15:30:59
61.170.179.89	30794	高匿名	HTTP	上海市 闸北/虹口区 电信	0.4秒	2018-12-11 14:29:31
117.87.176.212	9000	高匿名	HTTP	江苏省徐州市 电信	1秒	2018-12-11 13:30:56
42.59.87.104	1133	高匿名	HTTP	辽宁省葫芦岛市 联通	3秒	2018-12-11 12:30:46
180.118.135.67	9000	高匿名	HTTP	中国 江苏省 镇江市 电信	2秒	2018-12-11 11:30:58
112.86.0.106	53281	高匿名	HTTP	江苏省苏州市 联通	0.9秒	2018-12-11 10:30:57
114.234.83.35	9000	高匿名	HTTP	江苏省徐州市 电信	0.9秒	2018-12-11 09:31:00
60.191.201.38	45461	高匿名	HTTP	浙江省金华市 电信	3秒	2018-12-11 08:30:58
139.199.38.177	8118	高匿名	HTTP	北京市北京市 腾讯云计算（北京）有限责任公司 腾讯云	3秒	2018-12-11 07:30:43"""
proxyipinfo=[x.strip().split() for x in proxyipinfo.split('\n') if len(x)>0]
proxyiplist = [x[3].lower()+"://"+x[0]+":"+x[1] for x in proxyipinfo if x[0] != 'IP']
proxies = {'http':random.choice(proxyiplist)}
proxies.items()

dict_items([('http', 'http://61.170.179.89:30794')])

In [129]:
proxies = {'http':random.choice(proxyiplist)}
bcy=requests.get('https://bcy.net/',proxies=proxies)
bcy.status_code

200

# 验证IP是否可用

In [139]:
import telnetlib
ip=random.choice(proxyipinfo)[0]
port=random.choice(proxyipinfo)[1]
print(ip,port)
try:
    telnetlib.Telnet(ip, port=port, timeout=20)
except:
    print( 'connect failed')
else:
    print('success')


61.184.109.33 9000
connect failed


In [181]:
try:
    telnetlib.Telnet('117.48.209.178', port=4129, timeout=20)
except:
    print( 'connect failed')
else:
    print('success')


success


# IPIP网站获取自身IP

In [5]:
headers = {
        'user-agent': random.choice(user_agent_list)
    }
response = requests.get('https://www.ipip.net/ip.html',headers=headers)
doc=pq(response.text)
doc('table tr a').eq(0).text()

# 读取文件获取IP,PORT,TYPE

In [62]:
f_p = open('proxies.csv','r')
f_proxies=f_p.readlines()
proxies_usable=[]
for line in f_proxies:
    ip,host,protocol=line.split(',')
    if len(host)> 5:
        continue
    print(ip,host,protocol)
    proxies_usable.append([ip,host,protocol])
f_p.close()

223.100.166.3 36945 HTTP

180.118.134.156 9000 HTTP

115.223.230.66 9000 HTTP

183.147.28.186 9000 HTTP

49.70.209.158 9000 HTTP

115.218.217.81 9000 HTTP

115.223.228.54 9000 HTTP

183.129.207.81 12368 HTTP

115.223.228.48 9000 HTTP

111.230.254.195 8118 HTTP

211.103.198.106 8118 HTTP

121.232.148.60 9000 HTTP

183.166.129.53 8080 HTTP

111.160.236.84 40610 HTTP

47.96.148.248 8118 HTTP

223.100.166.3 36945 HTTP

180.118.134.156 9000 HTTP

115.223.230.66 9000 HTTP

183.147.28.186 9000 HTTP

49.70.209.158 9000 HTTP

115.218.217.81 9000 HTTP

115.223.228.54 9000 HTTP

183.129.207.81 12368 HTTP

115.223.228.48 9000 HTTP

111.230.254.195 8118 HTTP

211.103.198.106 8118 HTTP

121.232.148.60 9000 HTTP

183.166.129.53 8080 HTTP

111.160.236.84 40610 HTTP

47.96.148.248 8118 HTTP

119.57.108.109 53281 HTTP

119.57.156.90 53281 HTTP

163.125.16.194 8888 HTTP

114.249.113.126 9000 HTTP

113.116.145.136 9000 HTTP

101.231.50.154 8000 HTTP

113.65.160.148 9797 HTTP

115.171.203.68 9000 HTTP

113

In [83]:
testUrl = "https://www.ipip.net/ip.html/"
ip='221.178.176.25'
port='3128'
timeout=10
proxies = {"http":"http://"+ip+":"+port,"https":"http://"+ip+":"+port}
content=requests.get(testUrl,headers=headers,proxies=proxies,timeout=timeout)
